In [45]:
import pandas as pd
import numpy as np

In [55]:
# Jeu de données initial 
train = pd.read_json("train.json", lines = True)
test = pd.read_json("test.json", lines = True)

# Fichier output
sub = pd.read_csv("sample_submission.csv")

train

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,2395,id_ff84602f7,GGAAAAUAGCAGAGGAAAUACUAGAGCAAUUGCAAAGGCCGAUCAU...,........((..((......))...)).........(((..........,EEEEEEEESSIISSHHHHHHSSIIISSXXXXXXXXXSSSHHHHHHH...,4.036,1,

In [47]:
def divide_set(df, list_column):
    """
    Fonction qui crée d'une fenetre de lecture de 68 bases. 
        :Parameters:
            df = un data frame train
            list_column = numero de la colonne du data frame 
        :Return:
            deux data frame :
                first68_train = data frame contenant les 68 premieres bases et annotations de structure
                last68_train = data frame contenant les 68 dernieres bases et annotations de structure
    """
    first68 = df.copy()
    last68 = df.copy()
    for a in list_column :
        for i, r in enumerate(df.iloc[:,a]):
            first68.iloc[i,a] = r[0:68]
            last68.iloc[i,a] = r[len(r)-68:]
    return first68, last68

In [48]:
# Ces deux variables contiennent les 68 premières et dernières (respectivement), bases et informations de structure
# la colonne 3 correspond à l'information sur les appariements des bases
# la colonne 4 correspond à la prédiction des boucles

first68_train, last68_train = divide_set(train, [2,3,4])

In [49]:
print(len(first68_train.iloc[1,2]))
print(len(last68_train.iloc[1,2]))

68
68


### I) Préparation des données
##### a)  Nettoyage & préparation des données

In [50]:
# On commence par garder les séquences ARNm qui ont un rapport signal/noise >= 1.

print(len(first68_train))
first68_train = first68_train[first68_train['signal_to_noise'] >= 1]
print(len(first68_train))

2400
2097


In [51]:
# Création des formats fasta pour voir le pourcentage d'idendité avec CD-HIT web server 
# http://weizhong-lab.ucsd.edu/cdhit-web-server/cgi-bin/index.cgi?cmd=cd-hit
# Nous avons garder les paramètres par defaut

def fasta_file(f, df):
    """
    """
    with open(f,"w") as f_in :
        for line in df.iterrows():
            f_in.write(">" + str(line[1][1]) + "\n" + str(line[1][2]) + "\n")
            
            
fasta_file("train_fasta.fasta", train)
# ?????????????????????????????????????????????????????????????????????????????????????

In [56]:
def one_hot_encoding(df, name_colonne):
    """
    Fonction pour encoder les sequences, structures et type boucles : One hot encoding
        :Parameters:
            df = data frame : train et test de départ
            name_colonne = le nom de la colonne du df 
        :Return:
            un tableau
    """
    frst_lst = []
    for r in df[name_colonne]:
        for i in range(len(r)):
            if r[i] not in frst_lst:
                frst_lst.append(r[i])
    dico = {}
    ar = np.zeros(shape=(1,len(frst_lst)),dtype=int)
    for i, l in enumerate(frst_lst):
        ar2 = ar.copy()
        ar2[0][i]=1
        dico[l]=ar2
    scnd_lst = []
    for r in df[name_colonne]:
        lst = [] 
        for i in range(len(r)):
            #print(dico[r[i]], r[i])
            lst.append(dico[r[i]])
        scnd_lst.append(lst)
    return np.array(scnd_lst)

tab = one_hot_encoding(train, 'sequence')
tab2 = one_hot_encoding(train, 'structure')
tab3 = one_hot_encoding(train, 'predicted_loop_type')

train["One_hot seq"] = tab.tolist() # ajout des colonnes encoding a notre DF de départ
train["One hot strucure"] = tab2.tolist()
train["encoding predicted loop"] = tab3.tolist()
train

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,...,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,One_hot seq,One hot strucure,encoding predicted loop
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...",...,"[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000...","[[[1, 0, 0, 0]], [[1, 0, 0, 0]], [[0, 1, 0, 0]...","[[[1, 0, 0]], [[1, 0, 0]], [[1, 0, 0]], [[1, 0...","[[[1, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, ..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...",...,"[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[1, 0, 0, 0]], [[1, 0, 0, 0]], [[0, 1, 0, 0]...","[[[1, 0, 0]], [[1, 0, 0]], [[1, 0, 0]], [[1, 0...","[[[1, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, ..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...",...,"[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499...","[[[1, 0, 0, 0]], [[1, 0, 0, 0]], [[0, 1, 0, 0]...","[[[1, 0, 0]], [[1, 0, 0]], [[1, 0, 0]], [[1, 0...","[[[1, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, ..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...",...,"[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[1, 0, 0, 0]], [[1, 0, 0, 0]], [[0, 1, 0, 0]...","[[[1, 0, 0]], [[1, 0, 0]], [[1, 0, 0]], [[1, 0...","[[[1, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, ..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...",...,"[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,...","[[[1, 0, 0, 0]], [[1, 0, 0, 0]], [[0, 1, 0, 0]...","[[[1, 0, 0]], [[1, 0, 0]], [[1, 0, 0]], [[1, 0...","[[[1, 0,

In [59]:
# Vérification de la bonne taille de DF
print(len((one_hot_encoding(first68_train, 'sequence'))[0]))
print(len((one_hot_encoding(train, 'structure'))[0]))

68
107


In [73]:
# Boucle qui permet de determiner quelle base est apparié dans ce sens : (
dico_open = {}
for i, r in enumerate(first68_train.iterrows()):
    lst = []
    for j, l in enumerate(r[1][3]):
        if l == '(':
            lst.append(j)
    dico_open[str(i)+'('] = lst
dico_open
#len(dico)
# Nouveau dico, qui remplace les indices des bases par la base du dico_open précedent pour l'annotation de structure : (
new_dico_open = {}

for k in dico:
    lst = []
    for i in dico_open[k]:
        lst.append(first68_train.iloc[int(k.split('(')[0]),2][i])
    new_dico_open[k] = lst
new_dico_open

{'0(': ['A',
  'G',
  'C',
  'U',
  'C',
  'U',
  'G',
  'G',
  'G',
  'U',
  'U',
  'U',
  'C',
  'U',
  'A',
  'G'],
 '1(': ['G',
  'U',
  'G',
  'C',
  'C',
  'A',
  'G',
  'C',
  'U',
  'A',
  'G',
  'C',
  'U',
  'U',
  'C',
  'G',
  'A'],
 '2(': ['G',
  'G',
  'C',
  'C',
  'G',
  'C',
  'G',
  'C',
  'G',
  'G',
  'U',
  'A',
  'G',
  'C',
  'G',
  'G',
  'G',
  'C'],
 '3(': ['U',
  'U',
  'G',
  'C',
  'A',
  'U',
  'C',
  'G',
  'C',
  'G',
  'A',
  'C',
  'U',
  'C',
  'C',
  'A',
  'C',
  'A',
  'G',
  'C'],
 '4(': ['U',
  'C',
  'A',
  'U',
  'C',
  'G',
  'A',
  'G',
  'G',
  'A',
  'C',
  'G',
  'U',
  'C',
  'C',
  'G',
  'U',
  'U',
  'C',
  'A',
  'A',
  'C',
  'G',
  'C'],
 '5(': ['G',
  'C',
  'A',
  'G',
  'A',
  'C',
  'C',
  'G',
  'A',
  'U',
  'A',
  'C',
  'G',
  'U',
  'A',
  'G',
  'G',
  'A',
  'C',
  'C',
  'U'],
 '6(': ['G',
  'C',
  'G',
  'U',
  'C',
  'U',
  'A',
  'G',
  'G',
  'U',
  'A',
  'U',
  'A',
  'U',
  'G',
  'G',
  'A',
  'U',
  'G',
  'U',


In [71]:
# Boucle qui permet de determiner quelle base est apparié dans le sens : )
dico_close = {}
for i, r in enumerate(first68_train.iterrows()):
    lst = []
    for j, l in enumerate(r[1][3]):
        if l == ')':
            lst.append(j)
    dico_close[str(i)+')'] = lst

# Nouveau dico, qui remplace les indices des bases par la base du dico_close précedent pour l'annotation de structure : )
new_dico_close = {}
for k in dico_close:
    lst = []
    for i in dico_close[k]:
        lst.append(first68_train.iloc[int(k.split(')')[0]),2][i])
    new_dico_close[k] = lst
new_dico_close

{'0)': ['G',
  'G',
  'A',
  'G',
  'C',
  'U',
  'C',
  'U',
  'G',
  'G',
  'A',
  'A',
  'A',
  'C',
  'C',
  'C'],
 '1)': ['A',
  'G',
  'C',
  'U',
  'C',
  'G',
  'A',
  'U',
  'A',
  'G',
  'C',
  'U',
  'G',
  'G',
  'U',
  'A',
  'U'],
 '2)': ['C',
  'G',
  'C',
  'U',
  'A',
  'C',
  'C',
  'G',
  'C',
  'C',
  'G',
  'C',
  'G',
  'C',
  'G',
  'G',
  'C',
  'C'],
 '3)': ['G',
  'C',
  'U',
  'G',
  'U',
  'G',
  'G',
  'A',
  'G',
  'U',
  'C',
  'G',
  'C',
  'G',
  'A',
  'U',
  'G',
  'C',
  'A',
  'A'],
 '4)': ['G',
  'C',
  'G',
  'U',
  'U',
  'G',
  'A',
  'A',
  'C',
  'G',
  'G',
  'A',
  'C',
  'G',
  'U',
  'C',
  'C',
  'U',
  'C',
  'G',
  'A',
  'U',
  'G',
  'A'],
 '5)': ['A',
  'U',
  'C',
  'G',
  'G',
  'U',
  'C',
  'U',
  'G',
  'C',
  'A',
  'G',
  'G',
  'U',
  'U',
  'C',
  'U',
  'A',
  'C',
  'G',
  'U'],
 '6)': ['U',
  'U',
  'A',
  'U',
  'A',
  'A',
  'C',
  'A',
  'U',
  'C',
  'C',
  'A',
  'U',
  'G',
  'U',
  'A',
  'U',
  'U',
  'U',
  'A',


In [110]:
# Recherche des séquences qui n'ont pas le meme nombre de parenthèse ourverte et de parenthèse fermée.
for k in new_dico_open:
    if len(new_dico_open[k]) != len(new_dico_close[k.split('(')[0]+')']):
        print(k)
        print(len(new_dico_open[k]),len(new_dico_close[k.split('(')[0]+')']))

626(
12 11
1820(
27 14


In [120]:
# Code qui permet de donner le nombre de ( par base 

liste4 = []
tmp = -1
dty = {}
for k in new_dico:
    if (tmp != i) & (len(dty) > 0) :
        liste4.append(dty)
        dty = {}
    i = ''.join(x for x in k if x.isdigit())
    tmp = i
    for i in range(len(new_dico[k])):
        if new_dico[k][i] in dty:
            dty[new_dico[k][i]] += 1
        else :
            dty[new_dico[k][i]] = 1

liste4

[{'A': 2, 'G': 5, 'C': 3, 'U': 6},
 {'G': 5, 'U': 4, 'C': 5, 'A': 3},
 {'G': 10, 'C': 6, 'U': 1, 'A': 1},
 {'U': 4, 'G': 4, 'C': 8, 'A': 4},
 {'U': 5, 'C': 8, 'A': 5, 'G': 6},
 {'G': 6, 'C': 6, 'A': 6, 'U': 3},
 {'G': 9, 'C': 2, 'U': 9, 'A': 5},
 {'G': 7, 'C': 7, 'A': 4, 'U': 1},
 {'G': 3, 'U': 2, 'C': 1, 'A': 3},
 {'G': 7, 'C': 6, 'A': 5, 'U': 4},
 {'G': 9, 'C': 5, 'A': 2, 'U': 2},
 {'C': 5, 'G': 5, 'U': 5, 'A': 4},
 {'U': 4, 'C': 5, 'A': 4, 'G': 6},
 {'G': 7, 'C': 7, 'A': 3, 'U': 5},
 {'A': 1, 'U': 9, 'G': 5, 'C': 5},
 {'A': 3, 'G': 4, 'C': 3, 'U': 7},
 {'G': 6, 'C': 8, 'A': 6, 'U': 1},
 {'G': 7, 'C': 4, 'A': 3, 'U': 3},
 {'C': 6, 'G': 9, 'U': 4, 'A': 5},
 {'G': 7, 'U': 4, 'C': 4},
 {'G': 4, 'U': 5, 'A': 3, 'C': 3},
 {'U': 6, 'A': 1, 'C': 4, 'G': 4},
 {'C': 9, 'U': 4, 'G': 7, 'A': 4},
 {'A': 3, 'C': 7, 'G': 8, 'U': 3},
 {'C': 2, 'G': 8, 'U': 3, 'A': 3},
 {'C': 3, 'G': 2, 'A': 2, 'U': 1},
 {'C': 7, 'U': 8, 'A': 4, 'G': 4},
 {'G': 5, 'C': 5, 'A': 6, 'U': 3},
 {'G': 1, 'U': 1, 'A': 1, '

In [125]:
# Code qui permet de donner le nombre de ) par base

liste5 = []
tmp = -1
dty2 = {}
for k in new_dico1:
    if (tmp != i) & (len(dty2) > 0) :
        liste5.append(dty2)
        dty2 = {}
    i = ''.join(x for x in k if x.isdigit())
    tmp = i
    for i in range(len(new_dico1[k])):
        if new_dico1[k][i] in dty2:
            dty2[new_dico1[k][i]] += 1
        else :
            dty2[new_dico1[k][i]] = 1
liste5

[{'G': 5, 'A': 4, 'C': 5, 'U': 2},
 {'A': 4, 'G': 5, 'C': 3, 'U': 5},
 {'C': 10, 'G': 6, 'U': 1, 'A': 1},
 {'G': 8, 'C': 4, 'U': 4, 'A': 4},
 {'G': 8, 'C': 6, 'U': 5, 'A': 5},
 {'A': 3, 'U': 7, 'C': 5, 'G': 6},
 {'U': 9, 'A': 8, 'C': 5, 'G': 3},
 {'C': 7, 'A': 1, 'G': 7, 'U': 4},
 {'G': 1, 'A': 2, 'U': 4, 'C': 2},
 {'C': 7, 'G': 6, 'U': 5, 'A': 4},
 {'C': 9, 'G': 6, 'A': 1, 'U': 2},
 {'C': 5, 'G': 6, 'A': 4, 'U': 4},
 {'G': 6, 'U': 6, 'C': 4, 'A': 3},
 {'G': 8, 'U': 5, 'A': 4, 'C': 5},
 {'C': 5, 'A': 9, 'G': 5, 'U': 1},
 {'G': 6, 'A': 4, 'C': 4, 'U': 3},
 {'G': 8, 'C': 5, 'A': 1, 'U': 7},
 {'C': 6, 'G': 5, 'U': 4, 'A': 2},
 {'G': 6, 'C': 9, 'U': 5, 'A': 4},
 {'C': 6, 'A': 4, 'G': 4, 'U': 1},
 {'G': 4, 'A': 4, 'U': 4, 'C': 3},
 {'G': 6, 'C': 3, 'A': 4, 'U': 2},
 {'C': 6, 'G': 11, 'A': 2, 'U': 5},
 {'C': 6, 'G': 9, 'U': 5, 'A': 1},
 {'G': 3, 'C': 5, 'A': 2, 'U': 6},
 {'G': 3, 'A': 1, 'U': 2, 'C': 2},
 {'C': 4, 'G': 7, 'A': 8, 'U': 4},
 {'A': 2, 'C': 5, 'G': 6, 'U': 6},
 {'G': 1, 'U': 1, 

In [113]:
dico3 = {}

for k in new_dico:
    for i in range(len(new_dico[k])):
        if len(new_dico[k]) == len(new_dico1[k.split('(')[0]+')']):
            if str(k.split('(')[0])+new_dico[k][i]+new_dico1[k.split('(')[0]+')'][i] in dico3:
                dico3[str(k.split('(')[0])+new_dico[k][i]+new_dico1[k.split('(')[0]+')'][i]] += 1
            elif str(k.split('(')[0])+new_dico[k][i]+new_dico1[k.split('(')[0]+')'][i] in dico3:
                dico3[str(k.split('(')[0])+new_dico[k][i]+new_dico1[k.split('(')[0]+')'][i]] += 1
            else :
                dico3[str(k.split('(')[0])+new_dico[k][i]+new_dico1[k.split('(')[0]+')'][i]] = 1

dico3

{'0AG': 1,
 '0GG': 2,
 '0CA': 2,
 '0UG': 2,
 '0CC': 1,
 '0UU': 1,
 '0GC': 2,
 '0GU': 1,
 '0UA': 2,
 '0UC': 1,
 '0AC': 1,
 '1GA': 3,
 '1UG': 3,
 '1GC': 2,
 '1CU': 4,
 '1CC': 1,
 '1AG': 2,
 '1UA': 1,
 '1AU': 1,
 '2GC': 5,
 '2GG': 4,
 '2CC': 4,
 '2CU': 1,
 '2GA': 1,
 '2CG': 1,
 '2UG': 1,
 '2AC': 1,
 '3UG': 2,
 '3UC': 2,
 '3GU': 2,
 '3CG': 6,
 '3AU': 2,
 '3GA': 2,
 '3AC': 2,
 '3CA': 2,
 '4UG': 1,
 '4CC': 2,
 '4AG': 3,
 '4UU': 2,
 '4CU': 3,
 '4GG': 2,
 '4AA': 2,
 '4GA': 2,
 '4GC': 2,
 '4CG': 2,
 '4UC': 2,
 '4CA': 1,
 '5GA': 1,
 '5CU': 2,
 '5AC': 1,
 '5GG': 2,
 '5AG': 2,
 '5CC': 2,
 '5GU': 2,
 '5UC': 1,
 '5AA': 2,
 '5CG': 2,
 '5UU': 2,
 '5AU': 1,
 '5GC': 1,
 '6GU': 4,
 '6CU': 1,
 '6GA': 4,
 '6UU': 2,
 '6CA': 1,
 '6UA': 3,
 '6AC': 2,
 '6UC': 2,
 '6AU': 2,
 '6UG': 2,
 '6AG': 1,
 '6GC': 1,
 '7GC': 4,
 '7CA': 1,
 '7GG': 3,
 '7AU': 2,
 '7CC': 3,
 '7CU': 2,
 '7AG': 2,
 '7CG': 1,
 '7UG': 1,
 '8GG': 1,
 '8UA': 1,
 '8CU': 1,
 '8GU': 2,
 '8AA': 1,
 '8AC': 2,
 '8UU': 1,
 '9GC': 4,
 '9CG': 4,
 '9AU': 2,

In [114]:
lst_duo = []
for k in dico3:
    if tmp != i :
        lst_duo.append(dic)
        dic = {}
    i = ''.join(x for x in k if x.isdigit())
    tmp = i
    print(i)
    dic

0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
18
18
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
23
23
23
23
24
24
24
24
24
24
24
24
24
25
25
25
25
25
25
25
25
25
26
26
26
26
26
26
26
27
27
27
27
27
27
27
27
27
27
27
27
27
28
28
28
28
28
28
28
28
28
28
28
28
28
29
29
29
29
30
30
30
30
30
30
30
30
30
30
30
30
31
31
31
31
31
31
31
31
31
31
31
31
32
32
32
32
32
32
32
32
32
32
33
33
33
33
33
33
33
33
33
33
33
33
34
34
34
34
34
34
34
35
35
35
35
35
35
35
35
35
35
35
35
36
36
36
36
36
36
36
36
36
36
37
37
37
37
37
37
37
37
37

312
312
312
313
313
313
313
313
313
313
313
313
313
314
314
314
314
314
314
314
314
314
314
314
314
315
315
315
315
315
315
315
315
315
315
315
315
316
316
316
316
316
316
316
316
316
316
316
316
316
317
317
317
317
317
317
317
317
317
317
317
317
318
318
318
318
318
318
318
318
318
318
318
318
319
319
319
319
319
319
319
319
319
320
320
320
320
320
320
320
320
321
321
321
321
321
321
321
321
321
321
321
321
322
322
322
322
322
322
322
322
322
322
323
323
323
323
323
323
324
324
324
324
324
324
324
324
324
324
325
325
325
325
325
325
325
325
325
325
325
326
326
326
326
326
326
326
327
327
327
327
327
327
327
327
327
328
328
328
328
328
328
328
328
328
328
328
328
329
329
329
329
329
329
330
330
330
330
330
330
330
330
330
330
330
330
331
331
331
331
331
331
331
332
332
332
332
332
332
332
332
332
332
332
333
333
333
333
333
333
333
333
333
333
334
334
334
334
334
334
334
334
334
334
334
334
336
336
336
336
336
336
336
336
336
336
336
336
336
337
337
337
337
337
337
337
337
337
338
338


555
555
555
555
555
555
555
555
555
555
555
556
556
556
556
556
556
556
556
557
557
557
557
557
557
557
557
557
557
557
558
558
558
558
558
558
558
558
558
558
559
559
559
559
559
559
559
559
559
559
560
560
560
560
560
560
560
560
560
560
561
561
561
561
561
561
561
561
561
561
561
561
562
562
562
562
562
562
562
562
562
562
562
562
562
562
562
563
563
563
563
563
563
563
563
563
563
564
564
564
564
564
564
564
564
564
564
565
565
565
565
566
566
566
566
566
566
566
566
566
566
566
567
567
567
567
567
567
567
567
567
568
568
568
568
568
568
568
568
568
568
568
569
569
569
569
569
569
569
569
569
569
569
570
570
570
570
570
570
570
570
570
570
570
571
571
571
571
571
571
571
571
571
572
572
572
572
572
572
573
573
573
573
573
573
573
573
573
573
574
574
574
574
574
574
574
574
575
575
575
575
575
575
575
575
575
576
576
576
576
576
576
577
577
577
577
577
577
577
577
577
577
577
577
578
578
578
578
578
578
578
578
578
578
581
581
581
581
581
581
581
582
582
582
582
582
582
582
583
583


852
852
852
852
852
852
853
853
853
853
853
853
853
853
854
854
854
854
854
854
854
854
854
855
855
855
855
855
857
857
857
857
857
857
857
857
857
857
857
857
858
858
858
858
858
858
858
858
858
858
858
858
858
859
859
859
859
859
859
859
859
859
859
860
860
860
860
860
860
860
860
860
860
860
860
860
861
861
861
861
861
861
861
862
862
862
862
862
863
863
863
863
863
863
863
863
863
863
865
865
865
865
865
865
865
865
865
865
866
866
866
866
867
867
867
867
868
868
868
868
868
868
868
868
868
868
868
869
869
869
869
869
869
869
869
869
869
870
870
870
870
870
870
870
870
870
870
871
871
871
871
871
871
871
872
872
872
872
872
872
872
872
873
873
873
873
873
873
873
873
873
873
873
873
874
874
874
874
874
874
874
874
874
875
875
875
875
875
875
875
875
875
875
875
876
876
876
876
876
876
876
876
876
876
876
876
877
877
877
877
877
877
877
877
877
877
877
878
878
878
878
878
878
878
878
878
878
879
879
879
879
879
879
879
879
879
879
879
879
879
879
880
880
880
880
880
880
880
880
880


1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1157
1158
1158
1158
1158
1158
1158
1158
1158
1158
1158
1158
1158
1159
1159
1159
1159
1159
1159
1159
1159
1159
1159
1160
1160
1161
1161
1161
1161
1161
1161
1161
1161
1161
1161
1162
1162
1162
1162
1162
1162
1162
1162
1162
1163
1163
1163
1163
1163
1163
1163
1163
1163
1163
1163
1163
1163
1164
1164
1164
1164
1164
1164
1164
1164
1164
1164
1164
1164
1164
1165
1165
1165
1165
1166
1166
1166
1166
1166
1166
1166
1166
1166
1167
1167
1167
1167
1167
1167
1167
1167
1167
1167
1167
1168
1168
1168
1168
1168
1168
1168
1168
1168
1168
1168
1168
1170
1170
1170
1170
1170
1170
1170
1170
1170
1170
1170
1170
1171
1171
1171
1171
1173
1173
1173
1173
1173
1173
1173
1173
1174
1174
1174
1174
1174
1174
1174
1174
1174
1174
1174
1174
1174
1175
1175
1175
1175
1175
1175
1175
1175
1175
1175
1176
1176
1176
1176
1176
1176
1176
1176
1176
1176
1176
1176
1177
1177
1177
1177
1177
1177
1177
1177
1177
1177
1178
1178
1178
1178
1178
1178
1178
1179
1179
1179
1179
1179
1179


1401
1401
1401
1401
1401
1401
1401
1401
1401
1401
1401
1401
1402
1402
1402
1402
1402
1402
1402
1402
1402
1402
1402
1402
1403
1403
1403
1403
1403
1403
1403
1403
1403
1403
1404
1404
1404
1404
1405
1405
1405
1405
1405
1405
1405
1405
1405
1405
1405
1405
1405
1406
1406
1406
1406
1406
1406
1406
1407
1407
1407
1407
1407
1407
1407
1407
1407
1407
1407
1408
1408
1408
1408
1409
1409
1409
1409
1409
1409
1409
1409
1409
1409
1409
1409
1410
1410
1410
1410
1410
1410
1410
1410
1410
1410
1410
1411
1411
1411
1411
1411
1411
1411
1411
1411
1412
1412
1412
1412
1412
1412
1412
1412
1412
1412
1412
1412
1413
1413
1413
1413
1413
1413
1413
1413
1413
1413
1413
1414
1414
1414
1414
1414
1414
1414
1415
1415
1415
1415
1415
1415
1416
1416
1416
1416
1416
1416
1416
1416
1416
1416
1416
1417
1417
1417
1417
1417
1417
1417
1417
1418
1418
1418
1418
1418
1418
1418
1418
1418
1418
1419
1419
1419
1419
1419
1419
1419
1419
1419
1419
1419
1419
1420
1420
1420
1420
1420
1420
1420
1420
1420
1420
1420
1420
1421
1421
1421
1421
1421
1421


1654
1654
1654
1654
1654
1654
1655
1655
1655
1655
1655
1655
1655
1655
1655
1655
1655
1655
1655
1655
1656
1656
1656
1656
1656
1656
1656
1656
1656
1656
1656
1657
1657
1657
1657
1657
1657
1657
1657
1657
1657
1658
1658
1658
1658
1658
1658
1658
1658
1658
1658
1660
1660
1660
1660
1660
1660
1660
1660
1660
1660
1660
1661
1661
1661
1661
1661
1661
1661
1661
1661
1661
1661
1661
1662
1662
1662
1662
1662
1662
1662
1662
1662
1663
1663
1663
1663
1663
1663
1663
1663
1663
1663
1663
1663
1664
1664
1664
1664
1664
1664
1664
1665
1665
1665
1665
1665
1665
1665
1665
1665
1665
1665
1665
1666
1666
1666
1666
1666
1666
1666
1666
1666
1666
1666
1666
1668
1668
1668
1668
1668
1668
1668
1668
1668
1668
1669
1669
1669
1669
1669
1669
1669
1669
1669
1669
1669
1669
1670
1670
1670
1670
1670
1670
1670
1670
1670
1670
1671
1671
1671
1671
1671
1671
1671
1671
1671
1671
1671
1671
1671
1672
1672
1672
1672
1672
1672
1672
1672
1672
1672
1673
1673
1673
1673
1673
1673
1673
1673
1673
1674
1674
1674
1674
1674
1674
1674
1674
1674
1674


1901
1901
1901
1901
1901
1902
1902
1902
1902
1902
1902
1902
1902
1902
1903
1903
1903
1903
1903
1903
1903
1903
1904
1904
1904
1904
1904
1904
1904
1904
1904
1904
1905
1905
1905
1905
1905
1905
1905
1905
1905
1905
1905
1905
1906
1906
1906
1906
1906
1906
1906
1906
1906
1907
1907
1907
1907
1907
1907
1907
1907
1907
1907
1907
1909
1909
1909
1909
1909
1909
1909
1909
1909
1910
1910
1910
1910
1910
1910
1910
1910
1910
1910
1911
1911
1911
1911
1911
1911
1911
1911
1911
1911
1912
1912
1912
1912
1912
1912
1912
1912
1912
1912
1913
1913
1913
1913
1913
1913
1913
1913
1913
1913
1913
1913
1914
1914
1914
1914
1914
1914
1914
1914
1914
1914
1915
1915
1915
1915
1915
1915
1915
1915
1915
1916
1916
1916
1916
1916
1916
1916
1916
1916
1917
1917
1917
1917
1917
1917
1917
1917
1917
1917
1918
1918
1918
1918
1918
1918
1918
1918
1918
1920
1920
1920
1920
1920
1920
1920
1920
1920
1920
1920
1921
1921
1921
1921
1921
1921
1921
1922
1922
1922
1922
1922
1922
1922
1922
1922
1922
1922
1922
1923
1923
1923
1923
1923
1923
1923
1923


#### b) Analyse exploratoire des données

In [ ]:
# Faire de l’analyse exploratoire des données :
# faire une representation du signal_noise : expérience foirer ou pas ?
# Calculer le taux de chaque base de chaque séquence dans le train = 
# calcul du taux de bases apparié en faire une distribution ssf 
# calcul du taux de predicted loop => comment representer nos données ?
# corrélation entre variables régréssion linéaire ?? a voir 
# faire une moyenne du taux de degration de chaque arnm, sur les 2097, prendre les 100 premiers par exmple
# sur ces 100 premiers on cherche le type de boucle prédit
# on prend pour chaque séquence le taux de dégradation le plus élevée puis le + faible et 
# on regarde a quelle base/loop/appariement il correspond.